In [1]:
import numpy as np
from ase.io import read
import torch

In [2]:
frames=read('h2_equivariance/h2.xyz',':')

In [3]:
frames

[Atoms(symbols='H2', pbc=False),
 Atoms(symbols='H2', pbc=False),
 Atoms(symbols='H2', pbc=False),
 Atoms(symbols='H2', pbc=False),
 Atoms(symbols='H2', pbc=False)]

In [4]:
h=np.load('h2_equivariance/f_orbs/Ham.npz', allow_pickle=True)
ham=[h['arr_{}'.format(j)] for j in range(len(h))]

In [5]:
o=np.load('h2_equivariance/f_orbs/Ovl.npz', allow_pickle=True)
ovl=[o['arr_{}'.format(j)] for j in range(len(o))]

In [6]:
ovl[0]

array([[ 9.99999745e-01,  6.05240458e-07,  2.46723072e-06, ...,
         3.16587034e-17, -2.77555756e-17, -3.94649591e-17],
       [ 6.05240458e-07,  9.99999008e-01, -4.64405645e-06, ...,
        -3.88144378e-17, -1.84314369e-18,  2.42861287e-17],
       [ 2.46723072e-06, -4.64405645e-06,  9.99977246e-01, ...,
        -2.16840434e-18,  5.20417043e-18,  0.00000000e+00],
       ...,
       [ 3.16587034e-17, -3.88144378e-17, -2.16840434e-18, ...,
         1.00000001e+00, -1.25767452e-17,  1.73846565e-08],
       [-2.77555756e-17, -1.84314369e-18,  5.20417043e-18, ...,
        -1.25767452e-17,  1.00000025e+00,  6.93889390e-18],
       [-3.94649591e-17,  2.42861287e-17,  0.00000000e+00, ...,
         1.73846565e-08,  6.93889390e-18,  1.00000009e+00]])

In [7]:
ham[4].shape

(62, 62)

In [8]:

def _real2complex(L: int):
    """
    Computes the transformation matrix that goes from a set
    of real spherical harmonics, ordered as:
        (l, -l), (l, -l + 1), ..., (l, l - 1), (l, l)
    to complex-valued ones(following the Condon-Shortley phase convention)
    _complex2real is obtained simply by taking the conjugate transpose of the matrix.
    """
    I_SQRT_2 = 1.0 / np.sqrt(2)
    SQRT_2 = np.sqrt(2)
    dim = 2 * L + 1
    mat = torch.zeros((dim, dim), dtype=torch.complex128)
    # m = 0
    mat[L, L] = 1.0
    if L == 0:
        return mat
    for m in range(1, L + 1):
        # m > 0
        mat[L + m, L + m] = I_SQRT_2 * (-1) ** m
        mat[L + m, L - m] = I_SQRT_2 * 1j * (-1) ** m
        # m < 0
        mat[L - m, L + m] = I_SQRT_2
        mat[L - m, L - m] = -I_SQRT_2 * 1j
    return mat

def cart2sph(xyz):
    x,y,z = xyz[0], xyz[1], xyz[2]
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(XsqPlusYsq + z**2)               # r
    theta = np.arccos(z/r)     # theta
    if z==0 and x*y!=0:
        theta=np.pi/2
    phi = np.arctan2(y,x)                           # phi
#     if x<0 and y>=0:
#         phi= phi+ np.pi
#     elif x<0and y<0:
#         phi= phi- np.pi
    return r, theta, phi
# x = np.array([1/np.sqrt(2),0,1/np.sqrt(2)])


In [9]:
print(ovl[0][0,15:])
y_lm=torch.tensor(ovl[0][0,19:22], dtype=torch.cdouble)
print(y_lm)
y_lm1=torch.tensor(ovl[1][0,19:22], dtype=torch.cdouble)
print(y_lm1)

[ 1.38777878e-17  5.29212219e-10 -2.25514052e-17  6.07153217e-18
  1.56125113e-17  1.38777878e-17 -4.87838483e-08 -6.07153217e-18
  2.08166817e-17 -2.77555756e-17  2.60208521e-17  2.16840435e-17
  2.66974822e-08  8.67361738e-18 -1.73472348e-17  1.38777878e-17
  7.79055324e-01 -2.25018612e-01  4.87955733e-02 -1.65876365e-02
 -4.96780559e-03 -3.46944695e-18 -4.27528818e-01 -1.26851654e-17
  9.54097912e-18 -2.74255457e-01  1.90819582e-17  3.46944695e-18
  6.42459265e-02 -1.04083409e-17 -5.63785130e-18  2.08166817e-17
  2.12140080e-01  4.55364912e-17  2.57498016e-18  1.86482774e-17
 -2.86703712e-17 -1.07832116e-01  4.85722573e-17  2.08166817e-17
 -3.81639165e-17  1.21430643e-17  3.12250226e-17 -1.06350753e-01
  3.16587034e-17 -2.77555756e-17 -3.94649591e-17]
tensor([ 1.5613e-17+0.j,  1.3878e-17+0.j, -4.8784e-08+0.j],
       dtype=torch.complex128)
tensor([ 1.6046e-17+0.j, -1.9082e-17+0.j,  2.4392e-08+0.j],
       dtype=torch.complex128)


In [10]:
x = np.array([0,0,1])
L = 1
#y_lm = torch.tensor([sph_harm(m, L, phi, theta) for m in range(-L, L+1)])

r2c = {L: _real2complex(L) for L in range(5)}
c2r = {L: np.conjugate(r2c[L].T) for L in range(5)}
#r2c=
#c2r = (_real2complex(1).T).conj()
y_lm_c = r2c[1] @ y_lm1#_r
#y_lm_real = c2r[1] @ y_lm#_r
y_lm_r = c2r[1] @ y_lm_c


In [11]:
y_lm_c

tensor([ 1.7248e-08-1.1346e-17j, -1.9082e-17+0.0000e+00j,
        -1.7248e-08-1.1346e-17j], dtype=torch.complex128)

In [12]:
y_lm1

tensor([ 1.6046e-17+0.j, -1.9082e-17+0.j,  2.4392e-08+0.j],
       dtype=torch.complex128)

In [13]:
y_lm_r

tensor([ 1.6046e-17+0.j, -1.9082e-17+0.j,  2.4392e-08+0.j],
       dtype=torch.complex128)

# Rotation direction

In [14]:
def change_rotation_direction(matrices):
    ''' 
    Function that takes a dictionary of matrices in block form, and changes sperical harmonics convention
    '''
    from mlelec.utils.symmetry import _real2complex
    r2c = {L: _real2complex(L) for L in range(3)}
    c2r = {L: np.conjugate(r2c[L].T) for L in range(3)}

    H_inverse={}

    for shift in matrices.keys():

        blocks_inverse = []
        for i, (key,b) in enumerate(matrices[tuple(shift)].items()):
            x = b.values.type(torch.complex128)
            L = key['L']

            comp=torch.einsum('ij,sjp->sip',r2c[L].to(x), x).conj()
            factor=(-1)**torch.tensor(b.components[0]['M']).to(x)
            print(key,factor)
            comp2=torch.mul(comp,factor.unsqueeze(-1))#.unsqueeze(-1)
            p=torch.einsum('ij,sjp->sip',c2r[L].to(x), comp2)
            blocks_inverse.append(TensorBlock(values = np.real(p),#.cpu(),#b.values,#*((-1)**(lam)),
                                       components = b.components,
                                 samples = b.samples,#Labels(names = b.samples.names[:], values=np.asarray(b.samples.values)[:]),
                                       properties = b.properties)
                                )    

        H_inverse[tuple(shift)] = TensorMap(matrices[tuple(shift)].keys, blocks_inverse)    
    return H_inverse

In [29]:
def check_rotation(r1, rotations, precision=10**-5, output='sum'):
    '''This function takes a matrix r, rotates it by the angle 'rotations' provided. If the rotation direction is correct, the function should return no output. If it is wrong, the function returns the norm to the by 'rotations' matrix.
    
    '''

    from mlelec.utils.metatensor_utils import labels_where  
    from mlelec.utils.symmetry import _wigner_d_real
    import numpy as np
    from metatensor import Labels
    import torch

    z=0


    for j, (shift, stmap) in enumerate(r1.items()):
        for i, (k,b) in enumerate(stmap.items()):
            unrot_idx = labels_where(
                b.samples,
                selection=Labels(["structure"], values=np.asarray([[0]]).reshape(-1, 1)),
                return_idx=True,
            )[-1]
            rot_idx = labels_where(
                b.samples,
                selection=Labels(
                    ["structure"], values=np.asarray([[1]]).reshape(-1, 1)
                ),
                return_idx=True,
            )[-1]
            #print(k)
            L = k['L']
            li = k['l_i']
            lj = k['l_j']
            ni=k['n_i']
            nj=k['n_j']
            # L = k["spherical_harmonics_l"]
            #print(*rotations[1 -1])
            wd = _wigner_d_real(L, *rotations[1 -1]).to(b.values) #should be this after rotation
            #wd = _wigner_d_real(L, *rotations[0][::-1]*rot).to(b.values)
#             if L==0:
#                 c=torch.eye(1).cuda()
#             elif L==1:
#                 c=torch.eye(3).cuda()
#                 c[2][2]=-1
#             elif L==2:
#                 c=torch.eye(5).cuda()
#                 c[3][3]=-1
#             elif L==3:
#                 c=torch.eye(7).cuda()
#                 c[4][4]=-1
#                 c[6][6]=-1
#             elif L==4:
#                 c=torch.eye(9).cuda()
#                 c[3][3]=-1
#             #print(L)
#             wd=c@wd @c
            w=b.values[0].shape[0]
            cs=torch.tensor([(-1)**m if m>0 else 1 for m in range(int(-w+(w/2)),int(w-(w/2-1)))]).to(b.values)

            if (torch.linalg.norm(wd @ b.values[unrot_idx] - b.values[rot_idx]))>precision:
                if output=='full':
                    print(j,i,k.values,torch.linalg.norm(wd @ b.values[unrot_idx] - b.values[rot_idx]))
                    #print(b.values[unrot_idx],w)
                z+=torch.linalg.norm(wd @ b.values[unrot_idx] - b.values[rot_idx])
    print('Summed differences:',z)


In [30]:
print(ovl[0][0,15:])
y_lm=torch.tensor(ovl[0][0,19:22], dtype=torch.cdouble)
print(y_lm)
y_lm1=torch.tensor(ovl[1][0,19:22], dtype=torch.cdouble)
print(y_lm1)

[ 1.38777878e-17  5.29212219e-10 -2.25514052e-17  6.07153217e-18
  1.56125113e-17  1.38777878e-17 -4.87838483e-08 -6.07153217e-18
  2.08166817e-17 -2.77555756e-17  2.60208521e-17  2.16840435e-17
  2.66974822e-08  8.67361738e-18 -1.73472348e-17  1.38777878e-17
  7.79055324e-01 -2.25018612e-01  4.87955733e-02 -1.65876365e-02
 -4.96780559e-03 -3.46944695e-18 -4.27528818e-01 -1.26851654e-17
  9.54097912e-18 -2.74255457e-01  1.90819582e-17  3.46944695e-18
  6.42459265e-02 -1.04083409e-17 -5.63785130e-18  2.08166817e-17
  2.12140080e-01  4.55364912e-17  2.57498016e-18  1.86482774e-17
 -2.86703712e-17 -1.07832116e-01  4.85722573e-17  2.08166817e-17
 -3.81639165e-17  1.21430643e-17  3.12250226e-17 -1.06350753e-01
  3.16587034e-17 -2.77555756e-17 -3.94649591e-17]
tensor([ 1.5613e-17+0.j,  1.3878e-17+0.j, -4.8784e-08+0.j],
       dtype=torch.complex128)
tensor([ 1.6046e-17+0.j, -1.9082e-17+0.j,  2.4392e-08+0.j],
       dtype=torch.complex128)


In [31]:
from mlelec.utils.twocenter_utils_fhiaims import to_coupled_blocks,_orbs_offsets, _components_idx #, _matrix_to_condon_shortley

In [32]:
from typing import Union, List
import ase
from mlelec.utils.metatensor_utils import TensorBuilder

In [33]:
def _fhiaims_to_condon_shortley(
    matrices: Union[List[torch.tensor], torch.tensor],
    frames: Union[ase.Atoms, List[ase.Atoms]],
    orbitals: dict,
    device: str = None,
):
    # if not isinstance(frames, list):
    #     assert len(matrices.shape) == 2  # should be just one matrix (nao,nao)
    #     frames = [frames]
    #     matrices = matrices.reshape(1, *matrices.shape)

    orbs_tot, _ = _orbs_offsets(orbitals)

    block_builder = TensorBuilder(
        ["block_type", "species_i", "n_i", "l_i", "species_j", "n_j", "l_j"],
        ["structure", "center", "neighbor"],
        [["m1"], ["m2"]],
        ["value"],
    )    
    orbs_tot, _ = _orbs_offsets(orbitals)

    matrices_cs=torch.zeros(matrices.shape, dtype=torch.float64)
    print('cs',matrices_cs.shape)
    for A in range(len(frames)):
        frame = frames[A]
        ham = matrices[A]
        ki_base = 0
        for i, ai in enumerate(frame.numbers):
            kj_base = 0
            for j, aj in enumerate(frame.numbers):

                if isinstance(ham, np.ndarray):
                    block_data = torch.from_numpy(
                        ham[
                            ki_base : ki_base + orbs_tot[ai],
                            kj_base : kj_base + orbs_tot[aj],
                        ]
                    )
                    block_data2 = torch.from_numpy(
                        ham[
                            kj_base : kj_base + orbs_tot[aj],
                            ki_base : ki_base + orbs_tot[ai],

                        ]
                    )
                elif isinstance(ham, torch.Tensor):
                    block_data = ham[
                        ki_base : ki_base + orbs_tot[ai],
                        kj_base : kj_base + orbs_tot[aj],
                    ]
                    block_data2 = ham[
                        kj_base : kj_base + orbs_tot[aj],
                        ki_base : ki_base + orbs_tot[ai],
                    ]
                else:
                    raise ValueError

                ki_offset = 0
                for ni, li, mi in orbitals[ai]:
                    if (
                        mi != -li
                    ):  # picks the beginning of each (n,l) block and skips the other orbitals
                        continue
                    kj_offset = 0
                    for nj, lj, mj in orbitals[aj]:
                        if (
                            mj != -lj
                        ):  # picks the beginning of each (n,l) block and skips the other orbitals
                            continue
                        if ai == aj and (ni > nj or (ni == nj and li > lj)):
                            kj_offset += 2 * lj + 1
                            continue


                        islice = slice(ki_offset, ki_offset + 2 * li + 1)
                        jslice = slice(kj_offset, kj_offset + 2 * lj + 1)

                        w_i= 2 * li + 1
                        w_j= 2 * lj + 1
                        #print('wij',w_i,w_j)
                        cs_i=torch.tensor([(-1)**m if m>0 else 1 for m in range(int(-w_i+(w_i/2)),int(w_i-(w_i/2-1)))]).to(device)
                        #print(cs_i.T)
                        cs_j=torch.tensor([(-1)**m if m>0 else 1 for m in range(int(-w_j+(w_j/2)),int(w_j-(w_j/2-1)))]).to(device)
                        #print(cs_j)
                        #print(block_data[islice,jslice])
                        #print(block_data[islice,jslice]*cs_i.unsqueeze(-1)*cs_j)  


                        print(block_data.shape, ki_base, kj_base)
                        print(ki_base+ki_offset,ki_base+ki_offset+2 * li +1, kj_base+kj_offset,kj_base+kj_offset+2 * lj +1)
                        matrices_cs[A,ki_base+ki_offset:ki_base+ki_offset+2 * li +1, kj_base+kj_offset:kj_base+kj_offset+2 * lj +1]=block_data[islice,jslice]*cs_i.unsqueeze(-1)*cs_j
                        matrices_cs[A, kj_base+kj_offset:kj_base+kj_offset+2 * lj +1,ki_base+ki_offset:ki_base+ki_offset+2 * li +1]=block_data2[jslice,islice]*cs_i*cs_j.unsqueeze(-1)
                        
                        kj_offset += 2 * lj + 1
                    ki_offset += 2 * li + 1
                kj_base += orbs_tot[aj]

            ki_base += orbs_tot[ai]
    #return block_builder.build()
    return matrices_cs


In [34]:
# orbs={1:[[1,0,0],[2,0,0],[101,0,0],[102,0,0],
#         [103,1,-1],[103,1,0],[103,1,1],
#         [104,1,-1],[104,1,0],[104,1,1],
#         [105,2,-2],[105,2,-1],[105,2,0],[105,2,1],[105,2,2]]}
orbs={1:[[1,0,0],[102,0,0],[101,0,0],[111,0,0], [203,0,0],
        [102,1,-1],[102,1,0],[102,1,1],
        [112,1,-1],[112,1,0],[112,1,1],
        [203,1,-1],[203,1,0],[203,1,1],
        [103,2,-2],[103,2,-1],[103,2,0],[103,2,1],[103,2,2],
        [204,2,-2],[204,2,-1],[204,2,0],[204,2,1],[204,2,2],
        [204,3,-3],[204,3,-2],[204,3,-1],[204,3,0],[204,3,1],[204,3,2],[204,3,3]
        ]}


In [35]:
ham[0].shape

(62, 62)

In [36]:
b=np.array(ham)
m=_fhiaims_to_condon_shortley(b,frames,orbs)


cs torch.Size([5, 62, 62])
torch.Size([31, 31]) 0 0
0 1 0 1
torch.Size([31, 31]) 0 0
0 1 1 2
torch.Size([31, 31]) 0 0
0 1 2 3
torch.Size([31, 31]) 0 0
0 1 3 4
torch.Size([31, 31]) 0 0
0 1 4 5
torch.Size([31, 31]) 0 0
0 1 5 8
torch.Size([31, 31]) 0 0
0 1 8 11
torch.Size([31, 31]) 0 0
0 1 11 14
torch.Size([31, 31]) 0 0
0 1 14 19
torch.Size([31, 31]) 0 0
0 1 19 24
torch.Size([31, 31]) 0 0
0 1 24 31
torch.Size([31, 31]) 0 0
1 2 1 2
torch.Size([31, 31]) 0 0
1 2 3 4
torch.Size([31, 31]) 0 0
1 2 4 5
torch.Size([31, 31]) 0 0
1 2 5 8
torch.Size([31, 31]) 0 0
1 2 8 11
torch.Size([31, 31]) 0 0
1 2 11 14
torch.Size([31, 31]) 0 0
1 2 14 19
torch.Size([31, 31]) 0 0
1 2 19 24
torch.Size([31, 31]) 0 0
1 2 24 31
torch.Size([31, 31]) 0 0
2 3 1 2
torch.Size([31, 31]) 0 0
2 3 2 3
torch.Size([31, 31]) 0 0
2 3 3 4
torch.Size([31, 31]) 0 0
2 3 4 5
torch.Size([31, 31]) 0 0
2 3 5 8
torch.Size([31, 31]) 0 0
2 3 8 11
torch.Size([31, 31]) 0 0
2 3 11 14
torch.Size([31, 31]) 0 0
2 3 14 19
torch.Size([31, 31]) 0 0
2

In [37]:
print(m[0][1,:])
print(b[0][1,:])
print(m.shape,b.shape)
#print(torch.linalg.norm(m-b))
print(m[0][0,:]-b[0][0,:])

tensor([ 4.4304e-02,  5.7864e-01, -6.6759e-01,  3.8909e-01,  3.4868e-01,
         1.2056e-16, -2.8919e-02, -7.2642e-17,  1.3531e-16,  5.4445e-02,
        -1.5266e-16,  1.7365e-15,  9.0471e-03,  2.7756e-17,  3.4694e-17,
        -9.3675e-16, -1.6824e-02, -4.7054e-17, -2.7756e-17,  5.5511e-17,
        -7.4940e-16, -4.5144e-02,  2.5674e-16, -6.9389e-17, -2.7756e-16,
         3.2743e-17,  1.7937e-15, -8.6537e-03, -1.2490e-16,  2.4286e-17,
         7.9797e-17, -1.2356e-01,  2.8793e-02,  1.6447e-02, -1.1688e-01,
        -9.5136e-02,  2.9057e-17, -7.8747e-02, -2.1424e-16, -3.8164e-17,
         1.0544e-01, -7.3292e-17, -1.5959e-16, -1.9520e-01, -3.1312e-16,
         1.0625e-17, -3.6776e-16,  3.1979e-01, -2.1511e-16, -2.7105e-17,
        -5.2042e-18,  2.9143e-16,  1.2268e-01, -6.3480e-17, -2.7756e-17,
        -3.6082e-16,  1.9082e-17, -5.9674e-16, -3.7949e-01,  1.8562e-16,
         6.7221e-18, -6.5919e-17], dtype=torch.float64)
[ 4.43041243e-02  5.78644506e-01 -6.67592982e-01  3.89088176e-01
  3

In [38]:
rotations=np.array([[0,np.pi/2,0],[0,np.pi/4,0],[np.pi/4,np.pi/4,0],[np.pi/2,0,0]])
print(rotations[1 -1])

[0.         1.57079633 0.        ]


In [39]:
m={(0,0,0):m}
blocks=to_coupled_blocks(m,frames,orbs)

In [40]:
check_rotation(blocks, rotations, precision=10**-10, output='full')





0 8 [  0   1   1   0   1 103   2   2] tensor(1.0537e-08, device='cuda:0')
0 10 [  0   1   1   0   1 204   3   3] tensor(1.0537e-08, device='cuda:0')
0 17 [  0   1 102   0   1 103   2   2] tensor(1.3171e-09, device='cuda:0')
0 19 [  0   1 102   0   1 204   3   3] tensor(6.5854e-10, device='cuda:0')
0 27 [  0   1 101   0   1 103   2   2] tensor(5.2684e-09, device='cuda:0')
0 29 [  0   1 101   0   1 204   3   3] tensor(5.2684e-09, device='cuda:0')
0 35 [  0   1 111   0   1 204   3   3] tensor(2.6342e-09, device='cuda:0')
0 38 [  0   1 203   0   1 204   2   2] tensor(6.5854e-10, device='cuda:0')
0 39 [  0   1 203   0   1 204   3   3] tensor(2.6342e-09, device='cuda:0')
0 43 [  0   1 102   1   1 102   1   2] tensor(2.9802e-08, device='cuda:0')
0 46 [  0   1 102   1   1 112   1   2] tensor(1.4140e-08, device='cuda:0')
0 49 [  0   1 102   1   1 203   1   2] tensor(2.5944e-08, device='cuda:0')
0 50 [  0   1 102   1   1 103   2   1] tensor(2.1073e-08, device='cuda:0')
0 51 [  0   1 102   1   1 

In [47]:
blocks[(0,0,0)][0].samples

Labels(
    structure  center  neighbor
        0        0        0
        0        1        1
        1        0        0
        1        1        1
        2        0        0
        2        1        1
        3        0        0
        3        1        1
        4        0        0
        4        1        1
)

In [48]:
rotations

array([[0.        , 1.57079633, 0.        ],
       [0.        , 0.78539816, 0.        ],
       [0.78539816, 0.78539816, 0.        ],
       [1.57079633, 0.        , 0.        ]])

In [41]:
L=7
#[(-1)**m if m>0 else 1 for m in range(int(-L-(L/2+1)),int(L-(L/2+1)))]
[(-1)**m if m>0 else 1 for m in range(int(-L+(L/2)),int(L-(L/2-1)))]

[1, 1, 1, 1, -1, 1, -1]

In [79]:
import numpy as np
import hickle
from mlelec.utils.symmetry import _wigner_d_real, _rotation_matrix_from_angles
from mlelec.utils.twocenter_utils import _to_blocks, _to_coupled_basis, fix_orbital_order
import torch
torch.set_default_dtype(torch.float64)

In [46]:
orbitals = {'sto-3g': {6: [[1,0,0],[2,0,0],[2,1,1], [2,1,-1],[2,1,0]]}, 
            'def2svp': {6: [[1,0,0],[2,0,0],[3,0,0],[2,1,1], [2,1,-1],[2,1,0], [3,1,1], [3,1,-1],[3,1,0], [3,2,-2], [3,2,-1],[3,2,0], [3,2,1],[3,2,2]]},
            'gthszvmolopt': {6: [[2, 0, 0], [2, 1, -1], [2, 1, 0], [2, 1, 1]]
                            }}

In [47]:
cs_phase = {}
for basis in orbitals:
    cs_phase[basis] = {}
    for s in orbitals[basis]:
        cs_phase[basis][s] = [np.int32((-1)**v[2]) for v in orbitals[basis][s]]

In [92]:
from ase.io import read
frames = read('/home/pegolo/Software/my_mlelec/examples/data/periodic/c2/cp2k/rotated/C2_rotated.xyz', ':')

In [93]:
rot_fock = []
for i in range(3):
    rot_fock.append(np.load(f'/home/pegolo/Software/my_mlelec/examples/data/periodic/c2/cp2k/rotated/realoverlap_{i}.npy', allow_pickle=True).item())

# ORBS = 'gthszvmolopt'
# frames = read('/home/pegolo/Software/my_mlelec/examples/data/periodic/c2/cp2k/rotated/C2_rotated.xyz', ':')
# for ifr in range(len(frames)):
#     line = np.concatenate([cs_phase[ORBS][s] for s in frames[ifr].numbers])
#     cs = np.outer(line, line)
#     for T in rot_fock[ifr]:
#         rot_fock[ifr][T] = rot_fock[ifr][T]*cs

# for ifr in range(len(frames)):
#     for T in rot_fock[ifr]:
#         rot_fock[ifr][T] = fix_orbital_order(rot_fock[ifr][T], frames[ifr], orbitals[ORBS]) #### TODO <<

In [94]:
angles = hickle.load('/home/pegolo/Software/my_mlelec/examples/data/water_rotated/rotations.hickle')

In [97]:
T = 1,0,0
tmap0 = _to_coupled_basis(_to_blocks(rot_fock[0][T].reshape(1,8,8), frames[:1], {6: [[2, 0, 0], [2, 1, -1], [2, 1, 0], [2, 1, 1]]}, NH = True))
tmap1 = _to_coupled_basis(_to_blocks(rot_fock[1][T].reshape(1,8,8), frames[:1], {6: [[2, 0, 0], [2, 1, -1], [2, 1, 0], [2, 1, 1]]}, NH = True))

In [101]:
for (k0, b0), (k1, b1) in zip(tmap0.items(), tmap1.items()):
    L = k0['L']
    w0 = _wigner_d_real(L, *angles[0])
    # assert torch.norm(b0.values[...,0]@w0.T - b1.values[...,0]) < 1e-7, torch.norm(b0.values[...,0]@w0.T - b1.values[...,0])
    print(k0.values, L, torch.norm(b0.values[...,0]@w0.T - b1.values[...,0]), torch.norm(b0.values[...,0]))

[0 6 2 0 6 2 0 0] 0 tensor(0.) tensor(0.1095)
[0 6 2 0 6 2 1 1] 1 tensor(1.1005e-14) tensor(0.2219)
[0 6 2 1 6 2 0 1] 1 tensor(1.1005e-14) tensor(0.2219)
[0 6 2 1 6 2 1 0] 0 tensor(1.2832e-15) tensor(0.0322)
[0 6 2 1 6 2 1 2] 2 tensor(4.1103e-15) tensor(0.2981)
[1 6 2 0 6 2 0 0] 0 tensor(1.1264e-09) tensor(0.2806)
[1 6 2 0 6 2 1 1] 1 tensor(1.0269e-09) tensor(0.3395)
[-1  6  2  0  6  2  1  1] 1 tensor(1.0772e-09) tensor(0.3132)
[1 6 2 1 6 2 0 1] 1 tensor(1.0269e-09) tensor(0.3395)
[-1  6  2  1  6  2  0  1] 1 tensor(1.0772e-09) tensor(0.3132)
[1 6 2 1 6 2 1 0] 0 tensor(1.3575e-09) tensor(0.1349)
[1 6 2 1 6 2 1 2] 2 tensor(1.4149e-09) tensor(0.3338)
[-1  6  2  1  6  2  1  1] 1 tensor(0.) tensor(0.)


tensor([[1.]], dtype=torch.float64)

In [115]:
var_fock = []
for i in range(3):
    var_fock.append(np.load(f'/home/pegolo/Software/my_mlelec/examples/data/periodic/c2/cp2k/graphene_supercell_variety/realoverlap_{i}.npy', allow_pickle=True).item())

In [116]:
var_fock[0][0,0,0][:4,:4]-var_fock[2][0,0,0][:4,:4]

array([[ 0.00000000e+00,  6.05311018e-17,  0.00000000e+00,
         0.00000000e+00],
       [-6.05311018e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [104]:
var_fock[1][0,0,0]


array([[ 1.00000000e+00,  5.45020119e-17,  0.00000000e+00, ...,
         3.65841498e-18,  0.00000000e+00,  6.43940208e-02],
       [-5.45020119e-17,  1.00000000e+00,  0.00000000e+00, ...,
         2.77453094e-02,  0.00000000e+00, -7.29461648e-18],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  2.77453094e-02,  0.00000000e+00],
       ...,
       [ 3.65841498e-18,  2.77453094e-02,  0.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.77453094e-02, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 6.43940208e-02, -7.29461648e-18,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])